# 데이터 그룹화 함수 이해하기
 
### 그룹화의 개념 이해하기
 
**split-apply-combine**

In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1': np.random.randn(5),
                   'data2': np.random.randn(5)})

In [5]:
df

data1     data2 key1 key2
0  0.538739 -0.537541    a  one
1  1.423290  0.556410    a  two
2 -1.441264  0.791174    b  one
3 -0.835970  1.163689    b  two
4  0.223716 -1.507753    a  one

예를 들어 df 상에서 "key1" 열의 값이 'a'인 행에 대해서만, 그리고 'b'인 행에 대해서만 "data1" 열 값의 평균을 각각 계산하고자 한다고 하자. 데이터 그룹화는 이런 작업을 쉽고 빠르게 할 수 있도록 하는 기능이다. 데이터 그룹화를 하기 위해서 Series 혹은 DataFrame에 대하여 .groupby() 함수를 사용한다.

In [6]:
grouped = df['data1'].groupby(df['key1'])
grouped.mean()

key1
a    0.728582
b   -1.138617
Name: data1, dtype: float64

위 코드를 해석하자면, df의 "data1" 열을 사용하여 통계량을 계산할 것인데, 이 때 "key1" 열의 값을 기준으로 데이터를 먼저 그룹화한 뒤, 이들 각각의 그룹에 대하여 "data1" 열의 통계량을 계산한다는 의미이다. 
 


pandas에서의 모든 그룹화의 흐름은 이렇게 흘러가는데, 이를 "split-apply-combine""이라는 키워드로 표현하기도 합니다. 그룹화하고자 하는 열의 값을 기준으로 데이터를 나누고(split), 각 그룹에 대한 통계 함수를 적용하여(apply), 최종적인 통계량을 산출한 결과를 하나로 통합하여 표시하는(combine) 과정을 거치기 때문이다.
 
"data1" 열에 대한 통계량을 산출하는데, 이 때 그룹화의 기준으로 "key1"과 "key2" 열의 값을 동시에 사용하고, 여기에 .mean() 함수를 적용하여 각 그룹에 대하여 평균을 계산한 결과를 얻을 수 있다.


In [7]:
means = df["data1"].groupby([df["key1"], df["key2"]]).mean()
means.unstack()

key2       one      two
key1                   
a     0.381228  1.42329
b    -1.441264 -0.83597

지금까지의 .groupby() 함수의 호출 방식은, 해당 함수를 호출하는 대상과 해당 함수의 인자로 들어가는 대상이 df["data1"]과 같이 모두 Series인 경우의 방식이라고 할 수 있다. .groupby() 함수는 DataFrame에 대해서도 적용할 수 있다.

In [8]:
df.groupby('key1').mean()

data1     data2
key1                    
a     0.728582 -0.496295
b    -1.138617  0.977432

In [9]:
df.groupby("key1").count()

data1  data2  key2
key1                    
a         3      3     3
b         2      2     2

DataFrame에 대한 그룹화 결과, 그룹화 기준이 되는 열을 제외한 나머지 모든 열에 대한 통계량이 계산되었다. "key1"과 "key2" 열을 동시에 기준으로 삼아 그룹화하는 경우에도, 동일한 방식으로 하면 된다.

In [9]:
df.groupby(["key1", "key2"]).mean()

data1     data2
key1 key2                    
a    one   0.035608  0.134834
     two  -0.073330 -0.711286
b    one   0.032147  2.568801
     two   2.570439  0.570486

In [10]:
df.groupby(["key1", "key2"]).count()

data1  data2
key1 key2              
a    one       2      2
     two       1      1
b    one       1      1
     two       1      1

In [11]:
df

data1     data2 key1 key2
0  0.423534  0.194632    a  one
1 -0.073330 -0.711286    a  two
2  0.032147  2.568801    b  one
3  2.570439  0.570486    b  two
4 -0.352317  0.075037    a  one

In [10]:
# 그룹화 후 특정한 열 값에 대해서만 통계량을 산출
df.groupby(["key1", "key2"])["data2"].mean()

key1  key2
a     one    -1.022647
      two     0.556410
b     one     0.791174
      two     1.163689
Name: data2, dtype: float64

### 그룹에 대해 반복문 사용하기
 
위에서는 그룹화를 수행한 직후의 결과물을 직접 확인할 수 없었는데, 그룹에 대해 반복문을 수행하면 그룹화된 결과물에 대한 확인이 가능
 

In [13]:
for name, group in df.groupby('key1'):
    print('names : ',name)
    print('group', group)

names :  a
group       data1     data2 key1 key2
0  0.538739 -0.537541    a  one
1  1.423290  0.556410    a  two
4  0.223716 -1.507753    a  one
names :  b
group       data1     data2 key1 key2
2 -1.441264  0.791174    b  one
3 -0.835970  1.163689    b  two


df.groupby("key1")을 실행한 결과를 반복문에서 순회할 대상으로 명시한 뒤, name과 group 변수에서 값을 받도록 한 뒤 이들 값을 출력하면, 그룹화 기준 열의 값과 그룹화된 결과물을 직접 확인할 수 있다.

In [18]:
# 그룹화 기준 열이 2개인 경우
for (k1, k2), group in df.groupby(["key1", "key2"]):
    print('names : ',k1, k2)
    print(group)

names :  a one
      data1     data2 key1 key2
0  0.538739 -0.537541    a  one
4  0.223716 -1.507753    a  one
names :  a two
     data1    data2 key1 key2
1  1.42329  0.55641    a  two
names :  b one
      data1     data2 key1 key2
2 -1.441264  0.791174    b  one
names :  b two
     data1     data2 key1 key2
3 -0.83597  1.163689    b  two


그룹화 결과물을 반복문으로 보기 불편한 경우, 아예 딕셔너리 형태로 얻을 수도 있다.

In [19]:
pieces = dict(list(df.groupby("key1")))
pieces

{'a':       data1     data2 key1 key2
 0  0.538739 -0.537541    a  one
 1  1.423290  0.556410    a  two
 4  0.223716 -1.507753    a  one, 'b':       data1     data2 key1 key2
 2 -1.441264  0.791174    b  one
 3 -0.835970  1.163689    b  two}

### 딕셔너리 혹은 Series 기준으로 그룹화하기
 
그룹화를 수행하기 위한 기준으로, 별도로 정의된 딕셔너리 혹은 Series를 사용할 수 있다.
 

In [16]:
df2 = pd.DataFrame(np.random.randn(5, 5), 
                   columns=['a', 'b', 'c', 'd', 'e'],
                   index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
map_dict = {'a': 'red', 'b': 'red', 'c': 'blue', 
            'd': 'blue', 'e': 'red', 'f' : 'orange'}

In [17]:
df2

a         b         c         d         e
Joe    -1.249306 -2.089074  2.654914 -0.613105  0.669955
Steve   2.035816 -0.254109  0.945210 -0.844593 -0.269945
Wes    -1.230974  0.317459 -0.475854  0.055435  0.243665
Jim     1.030407  0.649253  0.267381  0.965723 -1.604467
Travis  0.746197 -0.789974  0.331719  0.148006  0.232730

In [18]:
map_dict

{'a': 'red', 'b': 'red', 'c': 'blue', 'd': 'blue', 'e': 'red', 'f': 'orange'}

df2를 생성할 때 map_dict라는 딕셔너리도 함께 생성하였는데, 이는 컬럼을 다른 문자열로 매핑하는 역할을 함. 해당 딕셔너리를 df.groupby() 함수의 인자로 넣게 되면, 해당 딕셔너리에 명시된 내용을 기준으로 인덱스 혹은 컬럼을 그룹화한다.

In [19]:
df2.groupby(map_dict, axis=1).sum()

blue       red
Joe     2.041809 -2.668424
Steve   0.100617  1.511762
Wes    -0.420419 -0.669850
Jim     1.233104  0.075193
Travis  0.479725  0.188952

위와 같이 실행하면, 'a'부터 'e'까지의 열을 map_dict에 명시된 내용대로 'red'와 'blue'로 그룹화한 뒤, 이들 내에서의 합계를 산출하여 보여준다. 딕셔너리 대신 Series를 사용하여 동일한 작업을 수행할 수도 있다.

In [20]:
map_s = pd.Series(map_dict)
df2.groupby(map_dict, axis=1).count()

blue  red
Joe        2    3
Steve      2    3
Wes        2    3
Jim        2    3
Travis     2    3

그룹화 결과 얻어진 그룹들에 대하여 적용할 수 있는 통계 함수들을 다음과 같이 정리하였다.
 
**그룹에 적용할 수 있는 주요 통계 함수**

- **count**  : 각 그룹 내의 (NaN이 아닌) 값들의 갯수를 계산 
- **sum** : 각 그룹 내의 (NaN이 아닌) 값들의 합을 계산 
- **mean** : 각 그룹 내의 (NaN이 아닌) 값들의 평균을 계산 
- **median** : 각 그룹 내의 (NaN이 아닌) 값들 중 중간값을 반환 
- **std, var** : 각 그룹 내의 값들의 표준편차, 분산을 계산 
- **min, max** : 각 그룹 내의 값들 중 최솟값, 최댓값을 반환 
- **prod** : 각 그룹 내의 (NaN이 아닌) 값들 전체의 곱을 계산 
- **first, last** : 각 그룹 내의 (NaN이 아닌) 값들 중 맨 첫번째, 맨 마지막 값을 반환 


### 그룹에 사용자 정의 함수 사용하기
 
그룹화 결과 얻어진 그룹에 대하여 사용자가 정의한 집계 함수를 사용할 수도 있다.
 

In [25]:
df

data1     data2 key1 key2
0  0.423534  0.194632    a  one
1 -0.073330 -0.711286    a  two
2  0.032147  2.568801    b  one
3  2.570439  0.570486    b  two
4 -0.352317  0.075037    a  one

"key1" 열의 값을 기준으로 그룹화한 뒤, 이를 grouped 변수에 저장. 그리고 다음과 같은 함수를 정의

In [20]:
grouped = df.groupby('key1')

In [21]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

peak_to_peak() 함수는 그룹 내 각 열의 최댓값에서 최솟값을 뺀 값을 계산합니다. 
아래와 같이 .agg() 함수를 사용하면, 그룹에 대하여 사용자가 정의한 집계 함수를 적용할 수 있다.

In [22]:
grouped.agg(peak_to_peak)

data1     data2
key1                    
a     1.199573  2.064163
b     0.605293  0.372515

한편 .describe() 함수를 사용하면 그룹에 대한 기본 통계량 계산 결과를 모두 보여준다

In [30]:
grouped.describe()

data1     data2
key1                          
a    count  3.000000  3.000000
     mean  -0.000705 -0.147206
     std    0.392991  0.492154
     min   -0.352317 -0.711286
     25%   -0.212824 -0.318125
     50%   -0.073330  0.075037
     75%    0.175102  0.134834
     max    0.423534  0.194632
b    count  2.000000  2.000000
     mean   1.301293  1.569644
     std    1.794844  1.413022
     min    0.032147  0.570486
     25%    0.666720  1.070065
     50%    1.301293  1.569644
     75%    1.935866  2.069222
     max    2.570439  2.568801